In [42]:
import json
import urllib
import pandas as pd
import sys
from urllib.request import urlopen
from pandas.io.json import json_normalize
import xml.etree.ElementTree as ET

# 마스크 판매 위치 가져옴 (판매량 데이터는 실시간 말고는 찾을 수가 없었음 ㅠ.ㅠ)

In [9]:
with urlopen('https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/stores/json?page=1&perPage=5000') as url :
    mask = url.read()

In [10]:
mask = json.loads(mask.decode('utf-8'))

In [19]:
temp = pd.DataFrame(mask)

In [22]:
json_struct = json.loads(temp.to_json(orient="records"))    
df_flat = pd.json_normalize(json_struct)

In [27]:
df_flat

,count,page,totalCount,totalPages,addr,store_code,lat,long,store_name,store_type
0,4905,1,26763,6,충청남도 홍성군 갈산면 상촌로 12-1,34840559,36.602986,126.548911,신신약국,01
1,4905,1,26763,6,대구광역시 달서구 달구벌대로 1736 1층 (두류동),37844571,35.856678,128.555278,밝은온누리약국,01
2,4905,1,26763,6,충청남도 보령시 남대천로 58 (대천동),34833323,36.348742,126.593534,지애약국,01
3,4905,1,26763,6,경상북도 고령군 쌍림면 대가야로 608,5038300744,35.679196,128.242253,고령쌍림우체국,02
4,4905,1,26763,6,부산광역시 금정구 서동로 168 송월빌딩 1층 102호 (서동),21839824,35.214448,129.104775,굿모닝약국,01
...,...,...,...,...,...,...,...,...,...,...
4900,4905,1,26763,6,"경기도 군포시 광정로 25-16 107호 (금정동, 퇴계종합상가)",31816860,37.355838,126.931762,유약국,01
4901,4905,1,26763,6,울산광역시 동구 명덕6길 16 C동 (서부동),38830329,35.523447,129.431600,위드팜대학약국,01
4902,4905,1,26763,6,전라남도 보성군 벌교읍 채동선로 244,36819417,34.843843,127.344650,정약국,01
4903,4905,1,26763,6,대구광역시 중구 중앙대로 359 2층 (남산동),37845063,35.865078,128.592770,메디약국,01


In [31]:
df_flat = pd.DataFrame()
for i in range(6) :
    with urlopen('https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/stores/json?page='+str(i+1)+'&perPage=5000') as url :
        mask = url.read()
    mask = json.loads(mask.decode('utf-8'))
    temp = pd.DataFrame(mask)
    json_struct = json.loads(temp.to_json(orient="records"))    
    if i == 0 :
        df_flat = pd.json_normalize(json_struct)
    else :
        new = pd.json_normalize(json_struct)
        df_flat = pd.concat([df_flat,new])

In [33]:
df_flat.tail()

,count,page,totalCount,totalPages,storeInfos.addr,storeInfos.code,storeInfos.lat,storeInfos.lng,storeInfos.name,storeInfos.type
1733,1738,6,26763,6,대구광역시 서구 달구벌대로 1805 (내당동),37834347,35.859820,128.562193,윤약국,01
1734,1738,6,26763,6,전라남도 여수시 신월로 712-1 (봉산동),36808415,34.734001,127.719918,남산약국,01
1735,1738,6,26763,6,경상북도 경산시 경안로38길 17 (중방동),37847082,35.825802,128.737720,경산제일약국,01
1736,1738,6,26763,6,부산광역시 동구 초량상로 93 1층 (초량동),21829420,35.120073,129.038783,백광약국,01
1737,1738,6,26763,6,"경기도 남양주시 홍유릉로248번길 39 122, 123호 (금곡동, 다남프라자)",31869475,37.627250,127.202376,신성약국,01


In [34]:
df_flat.columns = ['count','page','totalCount','totalPages','addr','store_code','lat','long','store_name','store_type']

In [35]:
df_flat.to_csv('mask_where.csv',encoding='utf-8')

## 코로나 관련 의료시설 : 시군구까지만 있고 위도경도는 없음

hosp_type : 국민안심병원 선정유형 (A: 일반 호흡기 환자 진료를 위한 호흡기 전용 외래 설치 운영 병원, B: 호흡기 환자 전용 외래입원 진료가 가능한 선별진료소 운영 병원)

code : 구분코드 (A0: 국민안심병원, 97: 코로나 검사실시기관, 99: 코로나 선별진료소 운영기관)
operable_date : 운영가능날짜

In [59]:
from bs4 import BeautifulSoup
import requests

In [60]:
url = 'http://apis.data.go.kr/B551182/pubReliefHospService/getpubReliefHospList?ServiceKey=FzUV7PUBcXXMk17rgEtB1%2FSBi58HuCFoIEKhy87Sn1uAYveJeOiykOVgmirP4p9mXuR9yOlf0YMUNgjDA2sgQw%3D%3D&pageNo=1&numOfRows=100'
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

In [74]:
operable_date = [x.text for x in soup.find_all('adtfrdd')]
hosp_type = [x.text for x in soup.find_all('hosptytpcd')]
sigungu = [x.text for x in soup.find_all('sggunm')]
sido = [x.text for x in soup.find_all('sidonm')]
code = [x.text for x in soup.find_all('spcladmtycd')]
tel = [x.text for x in soup.find_all('telno')]
yoyang = [x.text for x in soup.find_all('yadmnm')]

In [91]:
cn = ['operable_date','hosp_type','sigungu','sido','code','tel','name']

In [92]:
hospital = pd.DataFrame()
for i in range(11) :
    url = 'http://apis.data.go.kr/B551182/pubReliefHospService/getpubReliefHospList?ServiceKey=FzUV7PUBcXXMk17rgEtB1%2FSBi58HuCFoIEKhy87Sn1uAYveJeOiykOVgmirP4p9mXuR9yOlf0YMUNgjDA2sgQw%3D%3D&pageNo='+str(i+1)+'&numOfRows=100'
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    operable_date = [x.text for x in soup.find_all('adtfrdd')]
    hosp_type = [x.text for x in soup.find_all('hosptytpcd')]
    sigungu = [x.text for x in soup.find_all('sggunm')]
    sido = [x.text for x in soup.find_all('sidonm')]
    code = [x.text for x in soup.find_all('spcladmtycd')]
    tel = [x.text for x in soup.find_all('telno')]
    yoyang = [x.text for x in soup.find_all('yadmnm')]
    temp = pd.DataFrame(list(zip(operable_date,hosp_type,sigungu,sido,code,tel,yoyang)),columns = cn)
    hospital = pd.concat([hospital,temp])   
        

In [98]:
hospital = hospital.reset_index().drop('index',axis=1)

In [99]:
hospital.to_csv('hospital.csv',encoding='utf-8')